In [ ]:
# !pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
import pandas as pd
from collections import Counter
from tqdm.notebook import tqdm
import numpy as np
from scipy.spatial import distance
from tqdm.notebook import tqdm
from statistics import mean 

from natsort import natsorted
from matplotlib import pyplot as plt

from keras.utils import Sequence
import math
import pydicom
from keras.optimizers import Adam
import os
import keras.backend as K
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler

from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import ipywidgets as ipyw
import matplotlib.pyplot as plt
%matplotlib inline

class ImageSliceViewer3DMultipleColour:
    """ 
    ImageSliceViewer3D is for viewing volumetric image slices in jupyter or
    ipython notebooks. 
    
    User can interactively change the slice plane selection for the image and 
    the slice plane being viewed. 

    Argumentss:
    Volume = 3D input image
    figsize = default(8,8), to set the size of the figure
    cmap = default('plasma'), string for the matplotlib colormap. You can find 
    more matplotlib colormaps on the following link:
    https://matplotlib.org/users/colormaps.html
    
    """
    
    def __init__(self, volume, volume_1, volume_2, figsize=(8,8), cmap='plasma'):
        self.volume = volume/np.max(volume)
        self.volume_1 = volume_1/np.max(volume_1)
        self.volume_2 = volume_2/np.max(volume_2)
        self.figsize = figsize
        self.cmap = cmap
        self.v = [np.min(volume), np.max(volume)]
        
        # Call to select slice plane
        ipyw.interact(self.view_selection, view=ipyw.RadioButtons(
            options=['x-y','y-z', 'z-x'], value='x-y', 
            description='Slice plane selection:', disabled=False,
            style={'description_width': 'initial'}))
    
    def view_selection(self, view):
        # Transpose the volume to orient according to the slice plane selection
        orient = {"y-z":[1,2,0], "z-x":[2,0,1], "x-y": [0,1,2]}
        self.vol = np.transpose(self.volume, orient[view])
        self.vol_1 = np.transpose(self.volume_1, orient[view])
        self.vol_2 = np.transpose(self.volume_2, orient[view])
        maxZ = self.vol.shape[2] - 1
        
        # Call to view a slice within the selected slice plane
        ipyw.interact(self.plot_slice, 
            z=ipyw.IntSlider(min=0, max=maxZ, step=1, continuous_update=False, 
            description='Image Slice:'))
        
    def plot_slice(self, z):
        # Plot slice for the given plane and slice
        self.fig = plt.figure(figsize=self.figsize)
        
        img_true_mask = cv2.addWeighted(self.vol_1[:,:,z], 0.7, self.vol[:,:,z], 1, 0)
        img_pred_mask = cv2.addWeighted(self.vol_2[:,:,z], 0.7, self.vol[:,:,z], 1, 0)
#         final_image = np.hstack([self.vol[:,:,z], self.vol_1[:,:,z], self.vol_2[:,:,z]])
        final_image = np.hstack([self.vol[:,:,z], img_true_mask, img_pred_mask])
        self.ct_slice = self.vol[:,:,z]
        self.final_image = final_image
        self.img_true_mask = img_true_mask
        self.img_pred_mask = img_pred_mask
        self.z = z
        plt.imshow(final_image, cmap=plt.get_cmap(self.cmap))
#         return final_image

In [ ]:
def getMaxLength(arr):
 
    # intitialize count
    count = 0
     
    # initialize max
    result = 0
 
    for i in range(0, len(arr)):
     
        # Reset count when 0 is found
        if (arr[i] == 0):
            count = 0
 
        # If 1 is found, increment count
        # and update result if count  
        # becomes more.
        else:
             
            # increase count
            count+= 1
            result = max(result, count)  
         
    return result

In [ ]:
import ipywidgets as ipyw
import matplotlib.pyplot as plt
%matplotlib inline

class ImageSliceViewer3D:
    """ 
    ImageSliceViewer3D is for viewing volumetric image slices in jupyter or
    ipython notebooks. 
    
    User can interactively change the slice plane selection for the image and 
    the slice plane being viewed. 

    Argumentss:
    Volume = 3D input image
    figsize = default(8,8), to set the size of the figure
    cmap = default('plasma'), string for the matplotlib colormap. You can find 
    more matplotlib colormaps on the following link:
    https://matplotlib.org/users/colormaps.html
    
    """
    
    def __init__(self, volume, figsize=(8,8), cmap='plasma'):
        self.volume = volume
        self.figsize = figsize
        self.cmap = cmap
        self.v = [np.min(volume), np.max(volume)]
        
        # Call to select slice plane
        ipyw.interact(self.view_selection, view=ipyw.RadioButtons(
            options=['x-y','y-z', 'z-x'], value='x-y', 
            description='Slice plane selection:', disabled=False,
            style={'description_width': 'initial'}))
    
    def view_selection(self, view):
        # Transpose the volume to orient according to the slice plane selection
        orient = {"y-z":[1,2,0], "z-x":[2,0,1], "x-y": [0,1,2]}
        self.vol = np.transpose(self.volume, orient[view])
        maxZ = self.vol.shape[2] - 1
        
        # Call to view a slice within the selected slice plane
        ipyw.interact(self.plot_slice, 
            z=ipyw.IntSlider(min=0, max=maxZ, step=1, continuous_update=False, 
            description='Image Slice:'))
        
    def plot_slice(self, z):
        # Plot slice for the given plane and slice
        self.fig = plt.figure(figsize=self.figsize)
        plt.imshow(self.vol[:,:,z], cmap=plt.get_cmap(self.cmap))

In [ ]:
import ipywidgets as ipyw
import matplotlib.pyplot as plt
%matplotlib inline

class ImageSliceViewer3DMultiple:
    """ 
    ImageSliceViewer3D is for viewing volumetric image slices in jupyter or
    ipython notebooks. 
    
    User can interactively change the slice plane selection for the image and 
    the slice plane being viewed. 

    Argumentss:
    Volume = 3D input image
    figsize = default(8,8), to set the size of the figure
    cmap = default('plasma'), string for the matplotlib colormap. You can find 
    more matplotlib colormaps on the following link:
    https://matplotlib.org/users/colormaps.html
    
    """
    
    def __init__(self, volume, volume_1, volume_2, figsize=(8,8), cmap='plasma'):
        self.volume = volume/np.max(volume)
        self.volume_1 = volume_1/np.max(volume_1)
        self.volume_2 = volume_2/np.max(volume_2)
        self.figsize = figsize
        self.cmap = cmap
        self.v = [np.min(volume), np.max(volume)]
        
        # Call to select slice plane
        ipyw.interact(self.view_selection, view=ipyw.RadioButtons(
            options=['x-y','y-z', 'z-x'], value='x-y', 
            description='Slice plane selection:', disabled=False,
            style={'description_width': 'initial'}))
    
    def view_selection(self, view):
        # Transpose the volume to orient according to the slice plane selection
        orient = {"y-z":[1,2,0], "z-x":[2,0,1], "x-y": [0,1,2]}
        self.vol = np.transpose(self.volume, orient[view])
        self.vol_1 = np.transpose(self.volume_1, orient[view])
        self.vol_2 = np.transpose(self.volume_2, orient[view])
        maxZ = self.vol.shape[2] - 1
        
        # Call to view a slice within the selected slice plane
        ipyw.interact(self.plot_slice, 
            z=ipyw.IntSlider(min=0, max=maxZ, step=1, continuous_update=False, 
            description='Image Slice:'))
        
    def plot_slice(self, z):
        # Plot slice for the given plane and slice
        self.fig = plt.figure(figsize=self.figsize)
        
        final_image = np.hstack([self.vol[:,:,z], self.vol_1[:,:,z], self.vol_2[:,:,z]])
        
        plt.imshow(final_image, cmap=plt.get_cmap(self.cmap))

# 3D

In [ ]:
"""
V-Net Architecture
"""
import keras
import tensorflow as tf
# import keras.layers as keras_contrib 
import keras_contrib
# import tensorflow_addons as keras_contrib
# tfa





# Building blocks
def adding_conv(x, a, filters, kernel_size, padding, strides, data_format, groups):
    channel_axis = -1 if data_format=='channels_last' else 1
    c = keras.layers.Conv3D(filters, kernel_size, padding=padding, strides=strides, 
            activation=None, data_format=data_format)(x)
    c = keras.layers.add([c, a])
    c = keras_contrib.layers.GroupNormalization(groups=groups, axis=channel_axis)(c)
    c = keras.layers.advanced_activations.PReLU()(c)
    return c

def conv(x, filters, kernel_size, padding, strides, data_format, groups):
    channel_axis = -1 if data_format=='channels_last' else 1
    c = keras.layers.Conv3D(filters, kernel_size, padding=padding, strides=strides, 
            activation=None, data_format=data_format)(x)
    c = keras_contrib.layers.GroupNormalization(groups=groups, axis=channel_axis)(c)
    c = keras.layers.advanced_activations.PReLU()(c)
    return c

def down_conv(x, filters, kernel_size, padding, data_format, groups):
    channel_axis = -1 if data_format=='channels_last' else 1
    c = keras.layers.Conv3D(filters, kernel_size, padding=padding, strides=2, 
                            activation=None, data_format=data_format)(x)
    c = keras_contrib.layers.GroupNormalization(groups=groups, axis=channel_axis)(c)
    c = keras.layers.advanced_activations.PReLU()(c)
    return c

def up_conv_concat_conv(x, skip, filters, kernel_size, padding, strides, data_format, groups):
    channel_axis = -1 if data_format=='channels_last' else 1
    c = keras.layers.Conv3DTranspose(filters, kernel_size=(2,2,2), strides=(2,2,2), 
                                    data_format=data_format)(x) # up dim(x) by x2
    c = keras_contrib.layers.GroupNormalization(groups=groups, axis=channel_axis)(c)
    c = keras.layers.Conv3D(filters, kernel_size, padding=padding, strides=strides, 
                            activation=None, data_format=data_format)(c)
    concat = keras.layers.Concatenate(axis=channel_axis)([c, skip]) # concat after Up; dim(skip) == 2*dim(x)
    c = keras_contrib.layers.GroupNormalization(groups=groups, axis=channel_axis)(concat)
    c = keras.layers.advanced_activations.PReLU()(c)
    return c


# Encoders
def encoder1(x, filters, kernel_size, padding, strides, data_format, groups):
    with tf.variable_scope('encoder1'):
        with tf.variable_scope('conv'):
            conv1 = conv(x, filters, kernel_size, padding, strides, data_format, groups)
        with tf.variable_scope('addconv'):
            addconv = adding_conv(conv1, conv1, filters, kernel_size, padding, strides, data_format, groups) # N
        with tf.variable_scope('downconv'):
            downconv = down_conv(addconv, filters*2, kernel_size, padding, data_format, groups) # N/2
        return (addconv, downconv)

def encoder2(x, filters, kernel_size, padding, strides, data_format, groups):
    with tf.variable_scope('encoder2'):
        with tf.variable_scope('conv'):
            conv1 = conv(x, filters, kernel_size, padding, strides, data_format, groups) 
        with tf.variable_scope('addconv'):
            addconv = adding_conv(conv1, x, filters, kernel_size, padding, strides, data_format, groups) # N/2
        with tf.variable_scope('downconv'):
            downconv = down_conv(addconv, filters*2, kernel_size, padding, data_format, groups) # N/4
        return (addconv, downconv)

def encoder3(x, filters, kernel_size, padding, strides, data_format, groups):
    with tf.variable_scope('encoder3'):
        with tf.variable_scope('conv1'):
            conv1 = conv(x, filters, kernel_size, padding, strides, data_format, groups) # N/4
        with tf.variable_scope('conv2'):
            conv2 = conv(conv1, filters, kernel_size, padding, strides, data_format, groups) # N/4
        with tf.variable_scope('addconv'):
            addconv = adding_conv(conv2, x, filters, kernel_size, padding, strides, data_format, groups) # N/4
        with tf.variable_scope('downconv'):
            downconv = down_conv(addconv, filters*2, kernel_size, padding, data_format, groups) # N/8
        return (addconv, downconv)

def encoder4(x, filters, kernel_size, padding, strides, data_format, groups):
    with tf.variable_scope('encoder4'):
        with tf.variable_scope('conv1'):
            conv1 = conv(x, filters, kernel_size, padding, strides, data_format, groups) # N/8
        with tf.variable_scope('conv2'):
            conv2 = conv(conv1, filters, kernel_size, padding, strides, data_format, groups) # N/8
        with tf.variable_scope('addconv'):
            addconv = adding_conv(conv2, x, filters, kernel_size, padding, strides, data_format, groups) # N/8
        with tf.variable_scope('downconv'):
            downconv = down_conv(addconv, filters*2, kernel_size, padding, data_format, groups) # N/16
        return (addconv, downconv)


# Bottom
def bottom(x, filters, kernel_size, padding, strides, data_format, groups):
    with tf.variable_scope('bottom'):
        with tf.variable_scope('conv1'):
            conv1 = conv(x, filters, kernel_size, padding, strides, data_format, groups)
        with tf.variable_scope('conv2'):
            conv2 = conv(conv1, filters, kernel_size, padding, strides, data_format, groups)
        with tf.variable_scope('addconv'):
            addconv = adding_conv(conv2, x, filters, kernel_size, padding, strides, data_format, groups) # N/16
        return addconv # N/16


# Decoders
def decoder4(x, skip, filters, kernel_size, padding, strides, data_format, groups):
    with tf.variable_scope('decoder4'):
        with tf.variable_scope('upconv'):
            upconv = up_conv_concat_conv(x, skip, filters, kernel_size, padding, strides, data_format, groups) # N/8
        with tf.variable_scope('conv1'):
            conv1 = conv(upconv, filters, kernel_size, padding, strides, data_format, groups)
        with tf.variable_scope('conv2'):
            conv2 = conv(conv1, filters, kernel_size, padding, strides, data_format, groups)
        return conv2 # N/8

def decoder3(x, skip, filters, kernel_size, padding, strides, data_format, groups):
    with tf.variable_scope('decoder3'):
        with tf.variable_scope('upconv'):
            upconv = up_conv_concat_conv(x, skip, filters, kernel_size, padding, strides, data_format, groups) # N/4
        with tf.variable_scope('conv1'):
            conv1 = conv(upconv, filters, kernel_size, padding, strides, data_format, groups)
        with tf.variable_scope('conv2'):
            conv2 = conv(conv1, filters, kernel_size, padding, strides, data_format, groups)
        return conv2 # N/4

def decoder2(x, skip, filters, kernel_size, padding, strides, data_format, groups):
    with tf.variable_scope('decoder2'):
        with tf.variable_scope('upconv'):
            upconv = up_conv_concat_conv(x, skip, filters, kernel_size, padding, strides, data_format, groups) # N/2
        with tf.variable_scope('conv'):
            conv1 = conv(upconv, filters, kernel_size, padding, strides, data_format, groups)
        return conv1 # N/2

def decoder1(x, skip, filters, kernel_size, padding, strides, data_format, groups):
    with tf.variable_scope('decoder1'):
        with tf.variable_scope('upconv'):
            upconv = up_conv_concat_conv(x, skip, filters, kernel_size, padding, strides, data_format, groups) # N
        return upconv # N


# Attention gate
def attention_gate(inp, g, intra_filters):
    with tf.variable_scope('attention_gate'):
        data_format = 'channels_first'##@##
        groups = 8 ##@##

        # Gating signal processing
        g = keras.layers.Conv3D(intra_filters, kernel_size=1, data_format=data_format)(g) # N/2
        g = keras_contrib.layers.GroupNormalization(groups=groups, axis=1)(g) # N/2

        # Skip signal processing: 
        x = keras.layers.Conv3D(intra_filters, kernel_size=2, strides=2, padding='same', data_format=data_format)(inp) # N-->N/2
        x = keras_contrib.layers.GroupNormalization(groups=groups, axis=1)(x) # N

        # Add and proc
        g_x = keras.layers.Add()([g, x]) # N/2
        psi = keras.layers.Activation('relu')(g_x) # N/2
        psi = keras.layers.Conv3D(1, kernel_size = 1, padding='same', data_format=data_format)(psi) # N/2
        psi = keras_contrib.layers.GroupNormalization(groups=1, axis=1)(psi) # N/2
        psi = keras.layers.Activation('sigmoid')(psi) # N/2
        alpha = keras.layers.UpSampling3D(size=2, data_format=data_format)(psi) # N/2-->N


        x_hat = keras.layers.Multiply()([inp, alpha])
        return x_hat


# Model
def VNet(n_in, n_out, image_shape, filters, kernel_size, padding, strides, data_format, groups, inter_filters):
    with tf.variable_scope('VNet'):
        input_dim = image_shape+(n_in,) if data_format=='channels_last' \
            else (n_in,)+image_shape
        
        inputs = keras.layers.Input(input_dim)

        (encoder1_addconv, encoder1_downconv) = encoder1(inputs, filters*2**0, kernel_size, padding, strides, data_format, groups) # N, N/2
        (encoder2_addconv, encoder2_downconv) = encoder2(encoder1_downconv, filters*2**1, kernel_size, padding, strides, data_format, groups) # N/2, N/4
        (encoder3_addconv, encoder3_downconv) = encoder3(encoder2_downconv, filters*2**2, kernel_size, padding, strides, data_format, groups) # N/4, N/8
        (encoder4_addconv, encoder4_downconv) = encoder4(encoder3_downconv, filters*2**3, kernel_size, padding, strides, data_format, groups) # N/8, N/16

        bottom_addconv = bottom(encoder4_downconv, filters*2**4, kernel_size, padding, strides, data_format, groups) # N/16

        encoder4_ag = attention_gate(encoder4_addconv, bottom_addconv, inter_filters) # (N/8, N/16) --> N/8
        decoder4_conv = decoder4(bottom_addconv, encoder4_ag, filters*2**3, kernel_size, padding, strides, data_format, groups) # N/8
        encoder3_ag = attention_gate(encoder3_addconv, decoder4_conv, inter_filters) # (N/4, N/8) --> N/4
        decoder3_conv = decoder3(decoder4_conv, encoder3_ag, filters*2**2, kernel_size, padding, strides, data_format, groups) # N/4
        encoder2_ag = attention_gate(encoder2_addconv, decoder3_conv, inter_filters) # (N/2, N/4) --> N/2
        decoder2_conv = decoder2(decoder3_conv, encoder2_ag, filters*2**1, kernel_size, padding, strides, data_format, groups) # N/2
        encoder1_ag = attention_gate(encoder1_addconv, decoder2_conv, inter_filters) # (N, N/2) --> N
        decoder1_conv = decoder1(decoder2_conv, encoder1_ag, filters*2**0, kernel_size, padding, strides, data_format, groups) # N
       
        with tf.variable_scope("output"):
            outputs = keras.layers.Conv3D(n_out,
                (1,1,1),
                padding='same',
                activation='sigmoid',
                data_format=data_format)(decoder1_conv)

            model = keras.models.Model(inputs, outputs)

        return model

In [ ]:
"""
Data Generator for 3D Model
"""

from scipy import ndimage
import cv2
import random
class VolDataGenerator3D(Sequence):
    
    def __init__(self, ct_dataframe, unique_id_list, list_num_slices_list, batch_size, image_size, stack_size, overlap_size):
        self.unique_id_list = unique_id_list
        self.batch_size = batch_size
        self.image_size = image_size
        self.ct_dataframe = ct_dataframe
        self.stack_size = stack_size
        self.overlap_size = overlap_size
        self.batch_size = batch_size
        self.list_num_slices_list = list_num_slices_list
        
        assert(self.overlap_size < self.stack_size)
        random.seed(7)
        self.list_of_stacks = [(self.unique_id_list[i], current_stack) for i in range(len(self.unique_id_list)) for current_stack in self.make_overlapping_stacks(self.list_num_slices_list[i], self.stack_size, self.overlap_size) ]
#         random.shuffle(self.list_of_stacks)
        
    
    def __len__(self):
        return int(math.ceil(len(self.list_of_stacks)/self.batch_size))

    
    def rle_decode(self, mask_rle, shape):
        """
        Return an image array from run-length encoded string `mask_rle` with `shape`.
        """
        img = np.zeros(shape[0] * shape[1], dtype=np.uint)
        if mask_rle==[]:
            return np.zeros((shape[0], shape[1]), dtype=np.uint)
        else:
            s = mask_rle[0].split()
            starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
            starts -= 1
            ends = starts + lengths

            for low, up in zip(starts, ends): img[low:up] = 1
        return img.reshape(shape)

        
    def make_overlapping_stacks(self, num_slices, stack_size, overlap_size):
        list_of_indices = [(start, min(start + stack_size, num_slices - 1)) for start in range(0, num_slices, stack_size - overlap_size)]
        all_endings = list(map(lambda x: x[1], list_of_indices))
        first_index_of_max_end = all_endings.index(max(all_endings))
        list_of_indices = list_of_indices[: first_index_of_max_end + 1]
        self.list_of_indices = list_of_indices
        return list_of_indices


    def create_3d_vol(self, name_vs_num_slices_element):
#         name_vs_num_slices_element = ('img_0', (0, 100))
        self.name_vs_num_slices_element = name_vs_num_slices_element
        temp_df = self.ct_dataframe.loc[self.ct_dataframe["unique_identifier"] == name_vs_num_slices_element[0]]
        temp_df = temp_df.iloc[name_vs_num_slices_element[1][0] : name_vs_num_slices_element[1][1]]
        temp_df = temp_df.reset_index(drop=True)
        
        img_size = (pydicom.read_file(temp_df.iloc[0].dicom_path).pixel_array).shape  #SHAPE OF IMAGE
        
        img_vol = np.zeros((self.image_size[0], self.image_size[1],  self.stack_size))
        label_vol = np.zeros((self.image_size[0], self.image_size[1],  self.stack_size))
        
#         sum_pathos = 0
#         sum_pathos = temp_df.combine_pathos.sum()
        
        for i, row in temp_df.iterrows():
            img = (pydicom.read_file(row.dicom_path).pixel_array)
            img = cv2.resize(img, (self.image_size[0], self.image_size[1]))
            img_vol[:,:,i] = img
            
        unique_unique = (temp_df.combine_pathos.unique()).all()
        
#         if unique_unique != '[]':      #FOR OUTER CLASS TEST CSV
            
        for i, row in temp_df.iterrows():

            label_con = np.maximum(self.rle_decode(eval(row.consolidation), img_size), self.rle_decode(eval(row.predominant_consolidation), img_size))
            label_ggo = np.maximum(self.rle_decode(eval(row.ground_glass_opacity), img_size), self.rle_decode(eval(row.predominant_ground_glass_opacity), img_size))
            label_whole = np.maximum(label_con, label_ggo)
            label = cv2.resize(label_whole.astype(float), (self.image_size[0], self.image_size[1]))
            label_vol[:,:,i] = label
                
        self.temp_df = temp_df
        return img_vol, label_vol
            

    def create_volume_image(self, name_vs_num_slices_element):
        img_vol, label_vol = self.create_3d_vol(name_vs_num_slices_element)        
        
        img_vol = np.expand_dims(img_vol, 0)
        label_vol = np.expand_dims(label_vol, 0)
        return img_vol, label_vol


    def __getitem__(self, index):
        
        studies_for_this_batch = self.list_of_stacks[(self.batch_size*index):self.batch_size*(index+1)]
        self.studies_for_this_batch = studies_for_this_batch
        
        X = np.zeros((len(studies_for_this_batch), 1, self.image_size[0], self.image_size[1], self.stack_size))
        Y = np.zeros((len(studies_for_this_batch), 1, self.image_size[0], self.image_size[1], self.stack_size))
        
   
        for j,study in enumerate(studies_for_this_batch):
            X[j], Y[j] = self.create_volume_image(study)
        return X, Y

In [ ]:
"""
Finds Number of slices give the ID
"""

def find_num_slices_list(dataframe, scan_name_list):
    num_slices_list = []
    for name in scan_name_list:
        temp_df = dataframe.loc[dataframe.unique_identifier==name]
        num_slices_list.append(len(temp_df))
    return num_slices_list

In [ ]:
"""
Load the final CSV Made using the same procedure while training
"""
final_test_df_unique_series_id_list = (final_test_df.unique_identifier.unique())
final_test_df_unique_series_id_num_list = find_num_slices_list(final_test_df, final_test_df_unique_series_id_list)
assert(len(final_test_df_unique_series_id_list)==len(final_test_df_unique_series_id_num_list))

In [ ]:
"""
Data Generator Parameters
"""
image_size = (512, 512)
stack_size = 32
overlap_size = 0
batch_size = 1
assert(overlap_size < stack_size)


"""
Data Gen Object for whole test dataset
"""

final_test_df_gen_3D = VolDataGenerator3D(ct_dataframe = final_test_df,
                               unique_id_list = final_test_df_unique_series_id_list,
                               list_num_slices_list = final_test_df_unique_series_id_num_list,
                               batch_size = batch_size,
                               image_size = image_size,
                               stack_size = stack_size,
                               overlap_size = overlap_size)


"""
Data Gen Object for the Individual Scan
"""
x = 22              # Index of the CT scan you want to evaluate in the test set

temp_id_gen_3D = VolDataGenerator3D(ct_dataframe = final_test_df,
                               unique_id_list = final_test_df_unique_series_id_list[x:x+1],
                               list_num_slices_list = final_test_df_unique_series_id_num_list[x:x+1],
                               batch_size = batch_size,
                               image_size = image_size,
                               stack_size = stack_size,
                               overlap_size = overlap_size)

In [ ]:
"""
V-net imported
"""
image_shape = (image_size[0], image_size[1], stack_size)
group_size = 2
f_root = 8
filters = 4
model_3D = VNet(image_shape=image_shape, n_in=1, n_out=1, 
        strides=1, padding='same', kernel_size=3,
        groups=group_size, data_format='channels_first',
        inter_filters=f_root, filters = filters)

In [ ]:
"""Load the weight for the Model"""

model_checkpoint_path = '/opt/bucketdata/Users/...'
model_3D.load_weights(os.path.join(model_checkpoint_path, 'Weights', '....hdf5'))

In [ ]:
"""
Predictions for only one CT scan
"""
temp_id_pred_3D = model_3D.predict_generator(temp_id_gen_3D, verbose=1)

In [ ]:
"""
Predictions for the whole test dataset
"""
final_test_df_preds_3D = model_3D.predict_generator(final_test_df_gen_3D, verbose=1)

In [ ]:
"""
Side by side plotting of slice, label mask, and pred mask
"""

ImageSliceViewer3DMultipleColour(np.squeeze(temp_id_gen_3D[4][0]), np.squeeze(temp_id_gen_3D[4][1]), (np.squeeze(temp_id_pred_3D[4]>0.20)), cmap='gray', figsize=(8,8))

## POSITIVE PIXELS IN IMAGE:For temp ID INFERENCE

In [ ]:
"""
Making the true label volume for that scan
"""

temp_label_list = [(np.squeeze(temp_id_gen_3D[i][1])>0).astype(int) for i in range(len(temp_id_pred_3D))]
temp_label_vol_3D = np.dstack((temp_label_list))
temp_label_vol_3D.shape

In [ ]:
"""
Making the predicted masks volume for that scan
"""

temp_preds_list = [np.squeeze(temp_id_pred_3D[i]) for i in range(len(temp_id_pred_3D))]
temp_pred_vol_3D = np.dstack((temp_preds_list))
temp_pred_vol_3D.shape

In [ ]:
"""
Counting the number of positive pixels in each slice OR area of mask of Prediced 3D Volume
"""

threshold = 0.20       #Thresholding value
temp_pred_vol_3D = (temp_pred_vol_3D>threshold).astype(int)
pred_list_of_ones_3D = []
for pred in tqdm(range(temp_pred_vol_3D.shape[2])):
    pred = temp_pred_vol_3D[:,:,pred]
    counts_1_0 = Counter(pred.flatten())
    pred_list_of_ones_3D.append(counts_1_0[1])

In [ ]:
"""
Counting the number of positive pixels in each slice OR area of mask of the True label mask volume
"""

label_list_of_ones_3D = []
for label in tqdm(range(temp_label_vol_3D.shape[2])):
    label = temp_label_vol_3D[:,:,label]
    counts_1_0 = Counter(label.flatten())
    label_list_of_ones_3D.append(counts_1_0[1])

In [ ]:
"""
Plotting the normalised size of the masks for predicted and label 3D Volume
"""

normalised_pred_list_of_ones_3D = [i/max(pred_list_of_ones_3D) for i in pred_list_of_ones_3D]
plt.plot(normalised_pred_list_of_ones_3D)
plt.show()
normalised_label_list_of_ones_3D = [i/max(label_list_of_ones_3D) for i in label_list_of_ones_3D]
plt.plot(normalised_label_list_of_ones_3D)

## DICE COEFF: POSITIVE SLICES

In [ ]:
def dice_score(true,pred):
    """"
    Basiv Dice Score Function
    """
    return (1-(distance.dice(true.ravel(),pred.ravel())))


dice_scores_temp_list_3D=[]        #list of dice score of each slice in te CT Scan volume
for i in tqdm(range(temp_pred_vol_3D.shape[2])):
    score=dice_score(temp_label_vol_3D[:,:,i], temp_pred_vol_3D[:,:,i])
    dice_scores_temp_list_3D.append(score)

In [ ]:
#1 Approach
"""
Remove nan values and 0 values from the dice score list
"""

dice_scores_temp_list_pos_only_3D = [element for element in dice_scores_temp_list_3D if element!=0]
dice_scores_temp_list_pos_only_3D = [element for element in dice_scores_temp_list_pos_only_3D if not np.isnan(element)]
mean(dice_scores_temp_list_pos_only_3D)

In [ ]:
#2nd Approach
"""
Replace nan values with 1 (true negatives) and remove 0 values
"""
dice_scores_temp_list_pos_only_3D = []
for i in dice_scores_temp_list_3D:
    if np.isnan(i):
        dice_scores_temp_list_pos_only_3D.append(1)
    else:
        dice_scores_temp_list_pos_only_3D.append(i)
        
dice_scores_temp_list_pos_only_3D = [element for element in dice_scores_temp_list_pos_only_3D if element!=0]
mean(dice_scores_temp_list_pos_only_3D)

# 3D Inference, everything in a LOOP

In [ ]:
"""
Gives us the index of CTs which are positive
"""

final_test_df_unique_series_id_list_only_pos = []
for i in range(len(final_test_df_unique_id_list)):
    id_1_df = (final_test_df.loc[final_test_df.unique_identifier==final_test_df_unique_id_list[i]]).reset_index(drop=True)
    couer = Counter(id_1_df.true_label)
    if couer[1]!=0:
        final_test_df_unique_series_id_list_only_pos.append(i)
final_test_df_unique_series_id_list_only_pos

In [ ]:
"""
#LOOP FOR ALL THE CT SCANS TOGETHER
""""

normalised_pred_list_of_ones_3D_LIST = []
normalised_label_list_of_ones_3D_LIST = []

nan_not1butremoved_dice_scores_temp_list_pos_only_3D_nan_removed_LIST = []
nan_not1butremoved_dice_scores_temp_list_pos_only_3D_ZEROes_removed_LIST = []

for i in (final_test_df_unique_series_id_list_only_pos):
    temp_id_gen_3D = VolDataGenerator3D(ct_dataframe = final_test_df,
                               unique_id_list = final_test_df_unique_series_id_list[i:i+1],
                               list_num_slices_list = final_test_df_unique_series_id_num_list[i:i+1],
                               batch_size = batch_size,
                               image_size = image_size,
                               stack_size = stack_size,
                               overlap_size = overlap_size)
    
    temp_id_pred_3D = model_3D.predict_generator(temp_id_gen_3D, verbose=1)

    print("#1*****")
    temp_label_list = [(np.squeeze(temp_id_gen_3D[i][1])>0).astype(int) for i in range(len(temp_id_pred_3D))]
    temp_label_vol_3D = np.dstack((temp_label_list))
    
    print("#2*****")
    temp_preds_list = [np.squeeze(temp_id_pred_3D[i]) for i in range(len(temp_id_pred_3D))]
    temp_pred_vol_3D = np.dstack((temp_preds_list))
    
    print("#3*****")
    assert(temp_label_vol_3D.shape == temp_pred_vol_3D.shape)
    
    print("#4*****")
    temp_pred_vol_3D = (temp_pred_vol_3D>0.20).astype(int)
    pred_list_of_ones_3D = []
    for pred in tqdm(range(temp_pred_vol_3D.shape[2])):
        pred = temp_pred_vol_3D[:,:,pred]
        counts_1_0 = Counter(pred.flatten())
        pred_list_of_ones_3D.append(counts_1_0[1])
        
        
    print("#5*****")
    label_list_of_ones_3D = []
    for label in tqdm(range(temp_label_vol_3D.shape[2])):
        label = temp_label_vol_3D[:,:,label]
        counts_1_0 = Counter(label.flatten())
        label_list_of_ones_3D.append(counts_1_0[1])
        
    print("#6*****")
    normalised_pred_list_of_ones_3D = [i/max(pred_list_of_ones_3D) for i in pred_list_of_ones_3D]

    normalised_label_list_of_ones_3D = [i/max(label_list_of_ones_3D) for i in label_list_of_ones_3D]
    
    print("#7*****")
    
    
    normalised_pred_list_of_ones_3D_LIST.append(normalised_pred_list_of_ones_3D)
    normalised_label_list_of_ones_3D_LIST.append(normalised_label_list_of_ones_3D)
    
    print("#8*****")
    def dice_score(true,pred):
        return (1-(distance.dice(true.ravel(),pred.ravel())))

    dice_scores_temp_list_3D=[]
    for i in tqdm(range(temp_pred_vol_3D.shape[2])):
        score=dice_score(temp_label_vol_3D[:,:,i], temp_pred_vol_3D[:,:,i])
        dice_scores_temp_list_3D.append(score)
        
    print("#9*****")
    dice_scores_temp_list_pos_only_3D = []
    for i in dice_scores_temp_list_3D:
        if np.isnan(i):
#             dice_scores_temp_list_pos_only_3D.append(1)
            pass
        else:
            dice_scores_temp_list_pos_only_3D.append(i)

    print(mean(dice_scores_temp_list_pos_only_3D))
    nan_not1butremoved_dice_scores_temp_list_pos_only_3D_nan_removed_LIST.append(mean(dice_scores_temp_list_pos_only_3D))
    
    dice_scores_temp_list_pos_only_3D = [element for element in dice_scores_temp_list_pos_only_3D if element!=0]
    
    try:
        if mean(dice_scores_temp_list_pos_only_3D)==1:
            dice_scores_temp_list_pos_only_3D = [0]
        print(mean(dice_scores_temp_list_pos_only_3D))
    except:
        print("StatisticsError")
        dice_scores_temp_list_pos_only_3D = [0]
        
    nan_not1butremoved_dice_scores_temp_list_pos_only_3D_ZEROes_removed_LIST.append(mean(dice_scores_temp_list_pos_only_3D))
    
    del(temp_label_vol_3D)
    del(temp_pred_vol_3D)